In [1]:
import tensorflow as tf

In [2]:
from util.input import inputs, read_and_decode, aggregate_dataset

In [20]:
filenames = aggregate_dataset(['center'], ['original'], ['train'])['train']

In [21]:
filenames

['data/udacity-dataset-2/center/original/train_0.tfrecords']

In [22]:
num_epochs = None
batch_size = 100
one_hot_labels = False
with tf.name_scope('input'):
    filename_queue = tf.train.string_input_producer(
        filenames, num_epochs=num_epochs)

    image, angle, label, img_name = read_and_decode(filename_queue)
    if one_hot_labels:
        print("one_hot_labels")
        label = tf.one_hot(label, NUM_CLASSES, dtype=tf.int32)
        images, sparse_labels = tf.train.shuffle_batch(
            [image, label], batch_size=batch_size, num_threads=4,
            capacity=1000 + 3 * batch_size,
            # Ensures a minimum amount of shuffling of examples.
            min_after_dequeue=1000)
#         return images, sparse_labels
    else:
        pass

In [23]:
images, sparse_angles = tf.train.shuffle_batch(
                        [image, angle], batch_size=batch_size, num_threads=4,
                        capacity=1000 + 3 * batch_size,
                        # Ensures a minimum amount of shuffling of examples.
                        min_after_dequeue=1000)

In [30]:
images = []
angles = []

num_epochs = None
batch_size = 100
one_hot_labels = False
with tf.name_scope('input'):
    filename_queue = tf.train.string_input_producer(
        filenames, num_epochs=num_epochs)

    image, angle, label, img_name = read_and_decode(filename_queue)
    if one_hot_labels:
        print("one_hot_labels")
        label = tf.one_hot(label, NUM_CLASSES, dtype=tf.int32)
        images, sparse_labels = tf.train.shuffle_batch(
            [image, label], batch_size=batch_size, num_threads=4,
            capacity=1000 + 3 * batch_size,
            # Ensures a minimum amount of shuffling of examples.
            min_after_dequeue=1000)
#         return images, sparse_labels
    else:
        print('here')
        with tf.Session() as sess:
            # Start populating the filename queue.
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(coord=coord)

        #     images, sparse_angles = tf.train.shuffle_batch(
        #         [image, angle], batch_size=batch_size, num_threads=4,
        #         capacity=1000 + 3 * batch_size,
        #         # Ensures a minimum amount of shuffling of examples.
        #         min_after_dequeue=1000)

            for i in range(batch_size):
                # Retrieve a single instance:
                img = sess.run([image])
                print(img)
        #         if label == 0:

        #             if angle < -0.03966:
        #                 images.append(img)
        #                 angles.append(ang)
        #         elif label == 1:
        #             if -0.03966 <= ang and ang < -0.00698:
        #                 images.append(img)
        #                 angles.append(ang)
        #         elif label == 2:
        #             if -0.00698 <= ang and ang < 0.01266:
        #                 images.append(img)
        #                 angles.append(ang)
        #         elif label == 3:
        #             if 0.01266 <= ang and ang < 0.04189:
        #                 images.append(img)
        #                 angles.append(ang)
        #         elif label == 4:
        #             if 0.04189 <= ang:
        #                 images.append(img)
        #                 angles.append(ang)

            coord.request_stop()
            coord.join(threads)

here


InvalidArgumentError: Name: <unknown>, Feature: label.  Data types don't match. Data type: int64Expected type: float
	 [[Node: input_5/ParseSingleExample/ParseExample/ParseExample = ParseExample[Ndense=2, Nsparse=0, Tdense=[DT_STRING, DT_FLOAT], dense_shapes=[[], []], sparse_types=[], _device="/job:localhost/replica:0/task:0/cpu:0"](input_5/ParseSingleExample/ExpandDims, input_5/ParseSingleExample/ParseExample/ParseExample/names, input_5/ParseSingleExample/ParseExample/ParseExample/dense_keys_0, input_5/ParseSingleExample/ParseExample/ParseExample/dense_keys_1, input_5/ParseSingleExample/ParseExample/Const, input_5/ParseSingleExample/ParseExample/Const_1)]]

Caused by op 'input_5/ParseSingleExample/ParseExample/ParseExample', defined at:
  File "/Users/daniel/anaconda3/envs/tensorflow2/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/daniel/anaconda3/envs/tensorflow2/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_glob

In [18]:
images

<tf.Tensor 'shuffle_batch:0' shape=(100, 60, 80, 3) dtype=float32>

In [7]:
image, angle, label, img_name = read_and_decode(filename_queue)

NameError: name 'filename_queue' is not defined

In [ ]:
images, angles = inputs(train_dir, True, batch_size, num_batches, label, one_hot_labels=False)   